In [1]:
import pandas as pd

In [2]:
from sqlalchemy import create_engine
from config import database
from config import username
from config import password
from config import hostname

In [3]:
# Read in csv
babies = "finaldata/Table_4.1.csv"
babies_df = pd.read_csv(babies)
babies_df.head()

,babies_term,topic,topic_disaggregation,year,count,year_total,percent_total,lat,long
0,Pre-term,Admission to SCN/NICU,Admitted,2018,11839,31838,37.2,NaN,NaN
1,Pre-term,Admission to SCN/NICU,Not admitted,2018,2945,139691,2.1,NaN,NaN
2,Pre-term,Admission to SCN/NICU,Not stated,2018,33,1227,2.7,NaN,NaN
3,Term,Admission to SCN/NICU,Admitted,2018,19917,31838,62.6,NaN,NaN
4,Term,Admission to SCN/NICU,Not admitted,2018,136360,139691,97.6,NaN,NaN


In [4]:
# View all columns for df to decide what to keep
babies_df.columns

Index(['babies_term', 'topic', 'topic_disaggregation', 'year', 'count',
       'year_total', 'percent_total', 'lat', 'long'],
      dtype='object')

In [5]:
# Print 
babies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2646 entries, 0 to 2645
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   babies_term           2646 non-null   object 
 1   topic                 2646 non-null   object 
 2   topic_disaggregation  2646 non-null   object 
 3   year                  2646 non-null   int64  
 4   count                 2646 non-null   int64  
 5   year_total            2646 non-null   int64  
 6   percent_total         2646 non-null   float64
 7   lat                   320 non-null    float64
 8   long                  320 non-null    float64
dtypes: float64(3), int64(3), object(3)
memory usage: 186.2+ KB


In [6]:
# Check and change data types if required
babies_df.dtypes

babies_term              object
topic                    object
topic_disaggregation     object
year                      int64
count                     int64
year_total                int64
percent_total           float64
lat                     float64
long                    float64
dtype: object

In [7]:
# Count the total number of babies born between 2010 and 2018
total_births = babies_df["count"].sum()
total_births_df = pd.DataFrame ({"Number of Babies Born":[total_births]})

# Display the data frame
total_births_df

,Number of Babies Born
0,47552179


In [8]:
# Connect to postgres database
connection_string = (f'{username}:{password}@{hostname}:5432/mothersandbabies')
engine = create_engine(f'postgresql://{connection_string}')

In [9]:
# Check available tables in postgres database
engine.table_names()

['babies',
 'numberchildrenadopted',
 'adoptionsbytype',
 'mothers',
 'mothersbirthcountry',
 'birthlocation',
 'ageofbirthmother',
 'adoptionbyageandgender',
 'parentrelationship',
 'adoptiveparentrelationship',
 'typeofadoption',
 'intercountry_bycountryoforigin',
 'intercountry_byagegroup',
 'intercountry_bysiblinggroup',
 'adoptionprocesstime',
 'babiessexcount',
 'yearcount',
 'averageage',
 'birthstate',
 'termbabiescount']

In [10]:
# Load panda's dataframe to postgres sql table
babies_df.to_sql(name='babies', con=engine, if_exists='append', index=True)

In [11]:
# Count the total number of babies born per year based on term and sex
dropped = babies_df.loc[babies_df["topic"]== "Sex"]
dropped = dropped.loc[dropped["topic_disaggregation"]!= "Indeterminate/Not stated"]

birth_term = dropped.groupby(["year", "babies_term", "topic_disaggregation"])

babies_count = birth_term["count"].sum()

babies_count_df = pd.DataFrame ({"babies_count":babies_count})

# Display the data frame
babies_count_df.head(20)

babies_count
year babies_term topic_disaggregation              
2010 Not stated  Female                          20
                 Male                            43
     Post-term   Female                        1125
                 Male                          1235
     Pre-term    Female                       11455
                 Male                         13273
     Term        Female                      133861
                 Male                        138999
2011 Not stated  Female                          31
                 Male                            36
     Post-term   Female                         966
                 Male                          1122
     Pre-term    Female                       11679
                 Male                         13386
     Term        Female                      133985
                 Male                        140661
2012 Not stated  Female                          39
                 Male                            33
     Post-term   Female                         907
                 Male                           984

In [12]:
# Count the total number of babies born per year based on term and sex
dropped = babies_df.loc[babies_df["topic"]== "Sex"]
dropped = dropped.loc[dropped["topic_disaggregation"]!= "Indeterminate/Not stated"]

birth_term = dropped.groupby(["topic_disaggregation", "babies_term"])

babies_count = birth_term["count"].sum()

babies_count_df2 = pd.DataFrame ({"babies_count":babies_count})

# Display the data frame
babies_count_df2.head(20)

babies_count
topic_disaggregation babies_term              
Female               Not stated            500
                     Post-term            7924
                     Pre-term           121207
                     Term              1362081
Male                 Not stated            552
                     Post-term            8586
                     Pre-term           140954
                     Term              1428777

In [13]:
# Count the total number of babies born per year
peryear = babies_df.groupby(["year"])

babies_yearcount = peryear["count"].sum()

babies_yearcount_df = pd.DataFrame ({"babies_count":babies_yearcount})

# Display the data frame
babies_yearcount_df.head(20)

,babies_count
year,
2010,3760519
2011,3780238
2012,5045193
2013,5010371
2014,5062028
2015,5001371
2016,5096228
2017,4933955
2018,9862276


In [14]:
# Count the total number of babies born per year based on plurality
plural = babies_df.loc[babies_df["topic"]== "Plurality"]
pluraldropped = plural.loc[plural["topic_disaggregation"]!= "Not stated"]

byyear = pluraldropped.groupby(["year", "topic_disaggregation"])

plural_count = byyear["count"].sum()

plural_count_df = pd.DataFrame ({"babies_count":plural_count})

# Display the data frame
plural_count_df.head(30)

babies_count
year topic_disaggregation              
2010 Other multiples                222
     Singleton                   290773
     Twins                         9220
2011 Other multiples                228
     Singleton                   292736
     Twins                         9060
2012 Other multiples                191
     Singleton                   301025
     Twins                         9062
2013 Other multiples                249
     Singleton                   300148
     Twins                         9092
2014 Other multiples                219
     Singleton                   303207
     Twins                         9116
2015 Other multiples                237
     Singleton                   299728
     Twins                         8922
2016 Other multiples                  0
     Singleton                   305734
     Twins                         8818
2017 Other multiples                  0
     Singleton                   296579
     Twins                           32
2018 Other multiples                378
     Singleton                   585868
     Twins                        11402

In [15]:
# Count the total number of babies born per year based per state
babies_state = babies_df.loc[babies_df["topic"]== "State and territory of birth"]

state_per_year = babies_state.groupby(["year", "topic_disaggregation"])

state_count = state_per_year["count"].sum()

babiesbirthstate = pd.DataFrame ({"babies_count":state_count})

# Display the data frame
babiesbirthstate.head(30)

babies_count
year topic_disaggregation              
2010 ACT                           5946
     NSW                          96486
     NT                            3883
     QLD                          62025
     SA                           20001
     TAS                           6137
     VIC                          74472
     WA                           31265
2011 ACT                           5702
     NSW                          97238
     NT                            3927
     QLD                          62171
     SA                           20344
     TAS                           6323
     VIC                          74116
     WA                           32204
2012 ACT                           6144
     NSW                          99507
     NT                            4030
     QLD                          63709
     SA                           20666
     TAS                           5940
     VIC                          78393
     WA                           33862
2013 ACT                           6264
     NSW                          96968
     NT                            4061
     QLD                          63157
     SA                           20263
     TAS                           6021

In [16]:
# Count the total number of woman who gave birth in each state between 2010 and 2018
statedropped = babies_df.loc[babies_df["topic"]== "State and territory of birth"]

# Group by year and state
birth_state = statedropped.groupby(["year", "topic_disaggregation", "lat", "long"])

birth_state_count = birth_state["count"].sum()

birth_state_df_year = pd.DataFrame ({"count":birth_state_count})

# Display the data frame
birth_state_df_year

count
year topic_disaggregation lat        long              
2010 ACT                  -35.282001 149.128998    5946
     NSW                  -32.256943 148.601105   96486
     NT                   -12.462827 130.841782    3883
     QLD                  -27.470125 153.021072   62025
     SA                   -32.492500 137.765839   20001
...                                                 ...
2018 QLD                  -27.470125 153.021072  120966
     SA                   -32.492500 137.765839   38666
     TAS                  -42.880554 147.324997   11032
     VIC                  -36.757786 144.278702  157818
     WA                   -32.056946 115.743889   66854

[72 rows x 1 columns]

In [17]:
# Load panda's dataframe to postgres sql table
birth_state_df_year.to_sql(name='babiesbirthstate', con=engine, if_exists='append', index=True)

In [13]:
# Export file as a CSV, without the Pandas index, with the header
plural_count_df.to_csv("../machine_learning/termbabiescount.csv", index=False, header=True)

In [15]:
# Load panda's dataframe to postgres sql table
babies_count_df2.to_sql(name='babiessexcount', con=engine, if_exists='append', index=True)

In [29]:
# Load panda's dataframe to postgres sql table
babies_count_df.to_sql(name='termbabiescount', con=engine, if_exists='append', index=True)

In [30]:
# Export file as a CSV, without the Pandas index, with the header
babies_count_df.to_csv("../machine_learning/termbabiescount.csv", index=False, header=True)